In [2]:
import os

In [3]:
%pwd

'c:\\Users\\AQSA\\Desktop\\Machine-Learning-Project-with-MLflow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\AQSA\\Desktop\\Machine-Learning-Project-with-MLflow'

In [6]:
import dagshub
dagshub.init(repo_owner='ashpaqueshaikh4236', repo_name='machine-learning-project-with-mlflow', mlflow=True)

Accessing as ashpaqueshaikh4236

Initialized MLflow to track repo "ashpaqueshaikh4236/Machine-Learning-Project-with-MLflow"

Repository ashpaqueshaikh4236/Machine-Learning-Project-with-MLflow initialized!

In [45]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    target_column: str
    mlflow_uri: str

In [ ]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAM_FILE_PATH, schema_filepath = SCHEMA_FILE_PATH ):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])



    def get_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_path = config.metric_file_path,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/ashpaqueshaikh4236/Machine-Learning-Project-with-MLflow.mlflow"
        )
        
        return model_evaluation_config


In [47]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse
from src.mlproject.utils.common import save_json
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [48]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y,predicted_qualities)

            #saving metric as local
            scores = {"rmse":rmse, "mae":mae, "r2":r2}
            save_json(path = Path(self.config.metric_file_path),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            #Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [ ]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_evaluation_config()
    model_trainer_config = ModelEvaluation(config=model_evaluation_config)
    model_trainer_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-11-24 14:09:12,826: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-24 14:09:12,830: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-24 14:09:12,835: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-24 14:09:12,837: INFO: common: created directory at: artifacts]
[2024-11-24 14:09:12,838: INFO: common: created directory at: artifacts/model_evaluation]
[2024-11-24 14:09:12,929: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /ashpaqueshaikh4236/Machine-Learning-Project-with-MLflow.mlflow/api/2.0/mlflow/runs/create]
[2024-11-24 14:09:13,540: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/11/24 14:09:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/11/24 14:09:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


🏃 View run industrious-colt-805 at: https://dagshub.com/ashpaqueshaikh4236/Machine-Learning-Project-with-MLflow.mlflow/#/experiments/0/runs/89daca4bb52f434eb98738b87baaeaa3
🧪 View experiment at: https://dagshub.com/ashpaqueshaikh4236/Machine-Learning-Project-with-MLflow.mlflow/#/experiments/0
